In [ ]:
#Make a program to maximise sharpe
%pip install openpyxl

In [2]:
#Reading excel files into one
import openpyxl
import pandas as pd
import numpy as np
df = pd.read_csv('Stock2.csv')
returns = df.pct_change()
returns



,BAJAJ-AUTO,BAJAJ-FINSV,BAJAJ-FINANCE
0,NaN,NaN,NaN
1,-0.019181,0.089472,-0.031734
2,0.025289,0.155594,0.022294
3,-0.014306,-0.013340,-0.028332
4,-0.046292,0.018343,0.012469
...,...,...,...
3197,0.018067,0.008958,0.027129
3198,0.012477,0.039435,0.085477
3199,0.027539,0.065519,0.038620
3200,-0.013703,-0.012070,-0.006007


In [3]:
weights = np.array([0.3, 0.4, 0.3])

In [4]:
#Create cov matrix
cov_matrix_annual = returns.cov() * 252
cov_matrix_annual

,BAJAJ-AUTO,BAJAJ-FINSV,BAJAJ-FINANCE
BAJAJ-AUTO,0.114808,0.011991,0.018620
BAJAJ-FINSV,0.011991,0.160702,0.076005
BAJAJ-FINANCE,0.018620,0.076005,0.248456


In [5]:
#Create portfolio variance
port_variance = np.dot (weights.T, np.dot(cov_matrix_annual, weights))
port_variance

0.08287689040315016

In [6]:
#Calculate port volatility
port_volatility = np.sqrt(port_variance)
port_volatility

0.2878834667068433

In [7]:
#Calculate annual portfolio return
portfolioSimpleAnnualReturn = np.sum( returns.mean() * weights ) * 252
portfolioSimpleAnnualReturn

0.32291609386633363

In [8]:
#Show what we got
percent_var =  str (round (port_variance, 2) * 100) + '%'
percent_vols =  str (round (port_volatility, 2) * 100) + '%'
percent_ret =  str (round (portfolioSimpleAnnualReturn, 2) * 100) + '%'

print ('Expected annual return: ' + percent_ret)
print ('Annual risk: ' + percent_vols)
print ('Annual variance: ' + percent_var)

Expected annual return: 32.0%
Annual risk: 28.999999999999996%
Annual variance: 8.0%


In [10]:
%pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 7.2 MB/s 


In [11]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [16]:
#Portfolio Optimization
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

ef = EfficientFrontier(mu, S)
#ef.add_constraint(lambda x : x >= 0.3)

weights = ef.max_sharpe()
print ('Raw weights are: ', weights)
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose = True)

Raw weights are:  OrderedDict([('BAJAJ-AUTO', 0.3515420617272426), ('BAJAJ-FINSV', 0.4545236761063654), ('BAJAJ-FINANCE', 0.1939342621663921)])
Expected annual return: 23.1%
Annual volatility: 27.7%
Sharpe Ratio: 0.76


(0.2312810315046694, 0.27659145880033514, 0.7638740271339622)

In [42]:
#Checking for minimum 30 %
weightlist = []
# for  weight in weights.values :
#   weightlist.append(weight)
a = weights['BAJAJ-AUTO']
b = weights['BAJAJ-FINSV']
c = weights['BAJAJ-FINANCE']
print(a,b,c)


0.3515420617272426 0.4545236761063654 0.1939342621663921


In [43]:
#If 2 of the stocks are less than 30 then ratio of 3:3:4
weightlist = [a,b,c]
weightlist.sort()
if weightlist[1] < 0.3 : 
  weightlist = [0.3, 0.3, 0.4]


In [44]:
#If the first stock is less than 1
print(weightlist)
if weightlist[0]<0.3:
  wsum = weightlist[1]+weightlist[2]
  weightlist[0]= 0.3
  weightlist[1]= 0.7/wsum * weightlist[1]
  weightlist[2]= 0.7/wsum * weightlist[2]
  if weightlist [1] < 0.3 :
    weightlist = [0.3, 0.3, 0.4] 
print(wsum)
weightlist

[0.1939342621663921, 0.3515420617272426, 0.4545236761063654]
0.806065737833608


[0.3, 0.30528458369963213, 0.3947154163003679]

# New Section